### Importing libraries

In [1]:
import os
import pandas as pd

In [10]:
traffic_dfs = [pd.read_csv(os.path.join('data', f),
                           sep=';',
                           usecols=['Day',
                                    'Time',
                                    'Line',
                                    'Direction',
                                    'Stop name',
                                    'Stop number',
                                    'Passeng. in',
                                    'Passeng. out',
                                    'Tickets',
                                    'Pass. in [1]',
                                    'Pass. out [1]',
                                    'Pass. in [2]',
                                    'Pass. out [2]',
                                    'Pass. in [3]',
                                    'Pass. out [3]',
                                    'Pass. in [4]',
                                    'Pass. out [4]', ],
                           encoding='utf-8')
               for f in os.listdir('data') if f.endswith('csv') and not f.startswith('stops')]

traffic_df = pd.concat(traffic_dfs)
traffic_df['Passeng. in'] = traffic_df['Passeng. in'].fillna(0)
traffic_df['Passeng. out'] = traffic_df['Passeng. out'].fillna(0)
traffic_df['Pass. in [1]'] = traffic_df['Pass. in [1]'].fillna(0)
traffic_df['Pass. out [1]'] = traffic_df['Pass. out [1]'].fillna(0)
traffic_df['Pass. in [2]'] = traffic_df['Pass. in [2]'].fillna(0)
traffic_df['Pass. out [2]'] = traffic_df['Pass. out [2]'].fillna(0)
traffic_df['Pass. in [3]'] = traffic_df['Pass. in [3]'].fillna(0)
traffic_df['Pass. out [3]'] = traffic_df['Pass. out [3]'].fillna(0)
traffic_df['Pass. in [4]'] = traffic_df['Pass. in [4]'].fillna(0)
traffic_df['Pass. out [4]'] = traffic_df['Pass. out [4]'].fillna(0)

traffic_df['passengers_in'] = traffic_df['Passeng. in'] + \
                              traffic_df['Pass. in [1]'] + \
                              traffic_df['Pass. in [2]'] + \
                              traffic_df['Pass. in [3]'] + \
                              traffic_df['Pass. in [4]']

traffic_df['passengers_out'] = traffic_df['Passeng. out'] + \
                              traffic_df['Pass. out [1]'] + \
                              traffic_df['Pass. out [2]'] + \
                              traffic_df['Pass. out [3]'] + \
                              traffic_df['Pass. out [4]']

traffic_df['passengers_in'] = traffic_df['passengers_in'].astype('int64')
traffic_df['passengers_out'] = traffic_df['passengers_out'].astype('int64')

traffic_df['datetime'] = pd.to_datetime(traffic_df['Day'] + ' ' + traffic_df['Time'])

traffic_df = traffic_df.drop(['Passeng. in',
                              'Passeng. out',
                              'Pass. in [1]',
                              'Pass. out [1]',
                              'Pass. in [2]',
                              'Pass. out [2]',
                              'Pass. in [3]',
                              'Pass. out [3]',
                              'Pass. in [4]',
                              'Pass. out [4]',
                              'Day',
                              'Time',
                              'Tickets',
                              'Stop name'], axis=1)

traffic_df = traffic_df.rename(columns={'Direction': 'direction', 
                                        'Stop number': 'stop_number',
                                        'Line': 'line'})

pd.options.display.float_format = "{:.2f}".format

print(traffic_df.dtypes)
print(traffic_df.describe())

line                      object
direction                 object
stop_number                int64
passengers_in              int64
passengers_out             int64
datetime          datetime64[ns]
dtype: object


       stop_number  passengers_in  passengers_out
count   5529409.00     5529409.00      5529409.00
mean       1994.36           0.15            1.52
std        1735.79        1128.17         1702.94
min           0.00     -999996.00      -999997.00
25%         913.00           0.00            0.00
50%        1918.00           0.00            0.00
75%        2630.00           2.00            2.00
max       65534.00        1231.00      1000013.00


In [12]:
traffic_df = traffic_df[(traffic_df['passengers_in'] >= 0) & (traffic_df['passengers_in'] <= 20)]
traffic_df = traffic_df[(traffic_df['passengers_out'] >= 0) & (traffic_df['passengers_out'] <= 20)]

print(traffic_df.describe())

       stop_number  passengers_in  passengers_out
count   5484429.00     5484429.00      5484429.00
mean       1998.01           1.56            1.61
std        1737.52           2.81            2.92
min           0.00           0.00            0.00
25%         914.00           0.00            0.00
50%        1921.00           0.00            0.00
75%        2630.00           2.00            2.00
max       65534.00          20.00           20.00


In [8]:
def str_2_int(string, default=None):
    try:
        return int(string)
    except ValueError:
        return default


stop_df = pd.read_csv(os.path.join('data', 'stops_coordinates.csv'),
                      sep=',',
                      quotechar='"')

# Stop number field contains strings "1002a", "101b" and so on. 
# Those stops where removed, cause they never appeared in traffic data 
stop_df['Stop number'] = stop_df['Stop number'].apply(str_2_int)
stop_df = stop_df[stop_df['Stop number'].notnull()]

stop_df['Stop number'] = stop_df['Stop number'].astype('int64')

stop_df = stop_df.rename(columns={'Stop number': 'stop_number',
                                  'Stop name': 'stop_name',
                                  'Lng': 'lng',
                                  'Lat': 'lat'})

pd.options.display.float_format = "{:.2f}".format

print(stop_df.dtypes)
print(stop_df.describe())

stop_number      int64
stop_name       object
lng            float64
lat            float64
dtype: object
       stop_number     lng     lat
count      1408.00 1408.00 1408.00
mean       2432.26   25.27   54.69
std        1409.81    0.08    0.05
min         101.00   25.03   54.56
25%        1317.75   25.22   54.67
50%        2317.50   25.27   54.69
75%        3517.25   25.32   54.73
max        5300.00   25.46   54.83


In [13]:
df = pd.merge(traffic_df, stop_df, on='stop_number')

print(df.dtypes)
print(df.describe())

line                      object
direction                 object
stop_number                int64
passengers_in              int64
passengers_out             int64
datetime          datetime64[ns]
stop_name                 object
lng                      float64
lat                      float64
dtype: object


       stop_number  passengers_in  passengers_out        lng        lat
count   5197911.00     5197911.00      5197911.00 5197911.00 5197911.00
mean       2070.52           1.57            1.62      25.27      54.70
std        1293.07           2.81            2.91       0.06       0.04
min         101.00           0.00            0.00      25.09      54.58
25%        1020.00           0.00            0.00      25.23      54.68
50%        2007.00           0.00            0.00      25.27      54.70
75%        2704.00           2.00            2.00      25.30      54.73
max        5206.00          20.00           20.00      25.44      54.82
